Here we are going to try to train a multinomial MVPA classifier using logistic regression to recognize our different tasks. (Easy to repurpose this to recognize different task stimuli!) Let's see if the classifier accuracy improves over time! 

In [1]:
 from nilearn.input_data import NiftiMasker


In [2]:
# let's start off with day 1 data. We'll use a greymatter mask.
mask_filename = '/Volumes/cohen/abby/fMRIMultitasking/Subject_102/graymatter.nii'
fmri_filename = '/Volumes/cohen/abby/fMRIMultitasking/Subject_102/Multitasking_Subject102_Session1_20170505/cleanDay1_shft_aligned.nii'


In [ ]:
# easy way to mask fmri data
masker = NiftiMasker(mask_img=mask_filename, standardize=True)
fmri_masked = masker.fit_transform(fmri_filename)

In [3]:
# similar but slightly different way to mask fmri data
import nibabel 
mask = nibabel.load(mask_filename) 
mask_data = mask.get_data()
masker = NiftiMasker(mask_img=mask_filename, standardize=True)

In [ ]:
# I have super big epi files, so this takes a while
fmri_masked = masker.fit_transform(fmri_filename)

In [ ]:
# good to know what happened at each TR!
import scipy.io as sio
condition_label_filename = '/Users/abby/Dropbox/abigailnovick/multitaskingmultipurposing/fMRIMultitaskingLongTermCopy/SessionData/102/Session1ConditionLabels.mat'
matlabLabels = sio.loadmat(condition_label_filename)
labelArray = matlabLabels['conditionLabels']
maxes = np.max(labelArray, axis=0)
rest_mask = maxes == 0
labelVector_numbers = np.argmax(labelArray, axis=0) + 1
labelVector_numbers[rest_mask] = 0
# need to turn labels to strings
labelVector = labelVector_numbers.astype(str)

In [ ]:
# getting rid of rests
condition_mask = labelVector != "0"
fmri_masked_norest = fmri_masked[condition_mask]
condition_norest = labelVector[condition_mask]


In [ ]:
# one way to train classifier
import sklearn.svm
import sklearn.linear_model
svc_split = sklearn.linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg')


In [ ]:
# alternate way to train/test classifier
from sklearn.cross_validation import KFold
cv = KFold(n=len(fmri_masked_norest), n_folds=5)
target=condition_norest
for train, test in cv:
    svc_split.fit(fmri_masked_norest[train], target[train])
    prediction_split = svc_split.predict(fmri_masked_norest[test])
    print((prediction_split == target[test]).sum() / float(len(target[test])))


In [ ]:
# for new data, try next few sessions
new_data_predictions = svc_split.predict_log_proba(new_data)